<a href="https://colab.research.google.com/github/DAFNIKA/GUVI/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time

all_products = []
page_number = 1
records_to_collect = 12000
page_size = 1000


while len(all_products) < records_to_collect:
    url = f"https://world.openfoodfacts.org/api/v2/search?categories=chocolates&fields=code,product_name,brands,nutriments&page_size={page_size}&page={page_number}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        products = data.get('products', [])

        if not products:
            print(f"No more products found on page {page_number}. Stopping data collection.")
            break

        all_products.extend(products)

        print(f"Collected {len(all_products)} records so far (from page {page_number})")
        page_number += 1

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from page {page_number}: {e}")
        break
    except ValueError:
        print(f"Error decoding JSON from page {page_number}. Skipping this page.")
        page_number += 1

print(f"Finished collecting data. Total records collected: {len(all_products)}")


In [ ]:
import pandas as pd

df = pd.DataFrame(all_products)

df

In [ ]:
missing_counts = df.isnull().sum()
missing_percentage = (missing_counts / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Values': missing_counts,
    'Percentage (%)': missing_percentage
}).sort_values(by='Missing Values', ascending=False)

missing_df.head(20)

In [ ]:
threshold = 60
columns_to_drop = missing_df[missing_df['Percentage (%)'] > threshold].index

df_cleaned = df.drop(columns=columns_to_drop)

print(f"Dropped {len(columns_to_drop)} columns with > {threshold}% missing values.")

In [ ]:
categorical_cols = ['product_name', 'brands']
df_cleaned[categorical_cols] = df_cleaned[categorical_cols].fillna("Unknown")


In [ ]:
numerical_cols = df_cleaned.select_dtypes(include=['float64', 'int64']).columns

df_cleaned[numerical_cols] = df_cleaned[numerical_cols].fillna(df_cleaned[numerical_cols].median())


In [ ]:

df_cleaned.isnull().sum().sort_values(ascending=False).head(10)


In [ ]:
required_columns = {
    'energy-kcal_value': 'energy-kcal_100g',
    'energy-kj_value': 'energy_100g',
    'carbohydrates_value': 'carbohydrates_100g',
    'sugars_value': 'sugars_100g',
    'fat_value': 'fat_100g',
    'saturated-fat_value': 'saturated-fat_100g',
    'proteins_value': 'proteins_100g',
    'fiber_value': 'fiber_100g',
    'salt_value': 'salt_100g',
    'sodium_value': 'sodium_100g',
    'nova-group': 'nova_group',
    'nutrition-score-fr': 'nutrition-score-fr_100g',
    'fruits-vegetables-nuts-estimate-from-ingredients_100g': 'fruits-vegetables-nuts-estimate-from-ingredients_100g'
}

df_fe = df_cleaned.rename(columns=required_columns)

In [ ]:
nutrients_df = pd.json_normalize(df_fe['nutriments'])

df_fe = df_fe.join(nutrients_df)

df_fe['sugar_to_carb_ratio'] = df_fe['sugars_100g'] / df_fe['carbohydrates_100g']
df_fe['sugar_to_carb_ratio'] = df_fe['sugar_to_carb_ratio'].replace([float('inf'), -float('inf')], None)

In [ ]:
def classify_calories(kcal):
    if kcal < 150:
        return 'Low'
    elif 150 <= kcal <= 400:
        return 'Moderate'
    else:
        return 'High'

df_fe['calorie_category'] = df_fe['energy-kcal_100g'].apply(classify_calories)


In [ ]:
def classify_sugar(sugar):
    if sugar < 5:
        return 'Low Sugar'
    elif 5 <= sugar <= 15:
        return 'Moderate Sugar'
    else:
        return 'High Sugar'

df_fe['sugar_category'] = df_fe['sugars_100g'].apply(classify_sugar)


In [ ]:
df_fe['is_ultra_processed'] = df_fe['nova-group'].apply(lambda x: 'Yes' if x == 4 else 'No')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
print("Shape:", df_fe.shape)
df_fe.info()
df_fe.describe()


In [ ]:
missing = df_fe.isnull().sum().sort_values(ascending=False)
print(missing[missing > 0])


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(14, 10))

sns.histplot(df_fe['energy-kcal_100g'], bins=30, kde=True, ax=axs[0, 0]).set(title='Calories Distribution')
sns.histplot(df_fe['sugars_100g'], bins=30, kde=True, ax=axs[0, 1]).set(title='Sugars Distribution')
sns.histplot(df_fe['carbohydrates_100g'], bins=30, kde=True, ax=axs[1, 0]).set(title='Carbohydrates Distribution')
sns.histplot(df_fe['sugar_to_carb_ratio'], bins=30, kde=True, ax=axs[1, 1]).set(title='Sugar to Carb Ratio')

plt.tight_layout()
plt.show()


In [ ]:
sns.countplot(data=df_fe, x='calorie_category', palette='viridis').set(title='Calorie Category Count')
plt.show()

sns.countplot(data=df_fe, x='sugar_category', palette='magma').set(title='Sugar Category Count')
plt.show()


In [ ]:
df_fe['nova-group'].value_counts().plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6,6), title='NOVA Group Distribution')
plt.ylabel('')
plt.show()

In [ ]:
sns.countplot(data=df_fe, x='is_ultra_processed', palette='Set2').set(title='Ultra Processed Products')
plt.show()


In [ ]:
sns.scatterplot(data=df_fe, x='energy-kcal_100g', y='sugars_100g', hue='is_ultra_processed')
plt.title("Calories vs Sugars (Colored by Ultra-Processing)")
plt.show()


In [ ]:
plt.figure(figsize=(12, 8))
corr = df_fe.select_dtypes(include=['float64', 'int64']).corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.1f', square=True)
plt.title('Nutritional Correlation Heatmap')
plt.show()


In [ ]:
top_brands = df_fe.groupby('brands')[['energy-kcal_100g', 'sugars_100g']].mean().sort_values(by='energy-kcal_100g', ascending=False).head(10)

top_brands.plot(kind='bar', figsize=(10,6), title='Top 10 Brands by Avg Calories & Sugar')
plt.ylabel('Amount per 100g')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import sqlite3

conn = sqlite3.connect('chocolate_products1.db')
cursor = conn.cursor()


In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS product_info (
    product_code TEXT PRIMARY KEY,
    product_name TEXT,
    brand TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS nutrient_info (
    product_code TEXT PRIMARY KEY,
    energy_kcal_value FLOAT,
    energy_kj_value FLOAT,
    carbohydrates_value FLOAT,
    sugars_value FLOAT,
    fat_value FLOAT,
    saturated_fat_value FLOAT,
    proteins_value FLOAT,
    fiber_value FLOAT,
    salt_value FLOAT,
    sodium_value FLOAT,
    fruits_veg_nuts_estimate_100g FLOAT,
    nutrition_score_fr INTEGER,
    nova_group INTEGER,
    FOREIGN KEY (product_code) REFERENCES product_info(product_code)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS derived_metrics (
    product_code TEXT PRIMARY KEY,
    sugar_to_carb_ratio FLOAT,
    calorie_category TEXT,
    sugar_category TEXT,
    is_ultra_processed TEXT,
    FOREIGN KEY (product_code) REFERENCES product_info(product_code)
)
''')

conn.commit()


In [ ]:
df_product = df_fe[['code', 'product_name', 'brands']].rename(columns={
    'code': 'product_code',
    'brands': 'brand'
})

df_nutrients = df_fe[['code', 'energy-kcal_100g', 'energy_100g', 'carbohydrates_100g',
                      'sugars_100g', 'fat_100g', 'saturated-fat_100g', 'proteins_100g',
                      'fiber_100g', 'salt_100g', 'sodium_100g',
                      'fruits-vegetables-nuts-estimate-from-ingredients_100g',
                      'nutrition-score-fr_100g', 'nova-group']].rename(columns={
    'code': 'product_code',
    'energy-kcal_100g': 'energy_kcal_value',
    'energy_100g': 'energy_kj_value',
    'carbohydrates_100g': 'carbohydrates_value',
    'sugars_100g': 'sugars_value',
    'fat_100g': 'fat_value',
    'saturated-fat_100g': 'saturated_fat_value',
    'proteins_100g': 'proteins_value',
    'fiber_100g': 'fiber_value',
    'salt_100g': 'salt_value',
    'sodium_100g': 'sodium_value',
    'fruits-vegetables-nuts-estimate-from-ingredients_100g': 'fruits_veg_nuts_estimate_100g',
    'nutrition-score-fr_100g': 'nutrition_score_fr',
    'nova-group': 'nova_group'
})

df_derived = df_fe[['code', 'sugar_to_carb_ratio', 'calorie_category',
                    'sugar_category', 'is_ultra_processed']].rename(columns={
    'code': 'product_code'
})


In [ ]:
# Insert product info
for _, row in df_product.iterrows():
    cursor.execute('''
    INSERT OR IGNORE INTO product_info (product_code, product_name, brand)
    VALUES (?, ?, ?)
    ''', (row['product_code'], row['product_name'], row['brand']))

# Insert nutrient info
for _, row in df_nutrients.iterrows():
    cursor.execute('''
    INSERT OR IGNORE INTO nutrient_info (
        product_code, energy_kcal_value, energy_kj_value, carbohydrates_value,
        sugars_value, fat_value, saturated_fat_value, proteins_value,
        fiber_value, salt_value, sodium_value, fruits_veg_nuts_estimate_100g,
        nutrition_score_fr, nova_group
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(row))

# Insert derived metrics
for _, row in df_derived.iterrows():
    cursor.execute('''
    INSERT OR IGNORE INTO derived_metrics (
        product_code, sugar_to_carb_ratio, calorie_category,
        sugar_category, is_ultra_processed
    ) VALUES (?, ?, ?, ?, ?)
    ''', tuple(row))

conn.commit()


In [ ]:
cursor.execute('SELECT COUNT(*) FROM product_info')
print("Products in product_info:", cursor.fetchone()[0])

cursor.execute('SELECT COUNT(*) FROM nutrient_info')
print("Products in nutrient_info:", cursor.fetchone()[0])

cursor.execute('SELECT COUNT(*) FROM derived_metrics')
print("Products in derived_metrics:", cursor.fetchone()[0])


In [ ]:
df_product.to_csv('product_info.csv', index=False)
df_nutrients.to_csv('nutrient_info.csv', index=False)
df_derived.to_csv('derived_metrics.csv', index=False)


In [ ]:
print("df_product:", df_product.shape)
print("df_nutrients:", df_nutrients.shape)
print("df_derived:", df_derived.shape)


In [ ]:
print(df_fe.columns)


In [ ]:

df_fe['sugar_to_carb_ratio'] = df_fe['sugars_100g'] / df_fe['carbohydrates_100g']
df_fe['calorie_category'] = pd.cut(df_fe['energy-kcal_100g'],
                                   bins=[0, 200, 400, float('inf')],
                                   labels=['Low', 'Moderate', 'High'])
df_fe['sugar_category'] = pd.cut(df_fe['sugars_100g'],
                                 bins=[0, 10, 25, float('inf')],
                                 labels=['Low Sugar', 'Moderate Sugar', 'High Sugar'])
df_fe['is_ultra_processed'] = df_fe['nova-group'].apply(lambda x: 'Yes' if x == 4 else 'No')

In [ ]:
print(df_product.head())
print(df_nutrients.head())
print(df_derived.head())


In [ ]:
df_product.to_csv('product_info.csv', index=False)
df_nutrients.to_csv('nutrient_info.csv', index=False)
df_derived.to_csv('derived_metrics.csv', index=False)


In [ ]:
!pip install ipython-sql
%load_ext sql

In [ ]:
%sql sqlite:///chocolate_products1.db

In [ ]:
query = '''
SELECT brand, COUNT(*) AS product_count FROM product_info GROUP BY brand

'''
pd.read_sql(query, conn)


In [ ]:
query = '''
SELECT brand, COUNT(DISTINCT product_name) AS unique_product_count
FROM product_info
GROUP BY brand;


'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT brand, COUNT(*) AS product_count
FROM product_info
GROUP BY brand
ORDER BY product_count DESC
LIMIT 5;


'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT *
FROM product_info
WHERE product_name IS NULL OR product_name = '';
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT COUNT(DISTINCT brand) AS unique_brands
FROM product_info;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT *
FROM product_info
WHERE product_code LIKE '3%';
'''
pd.read_sql(query, conn)

In [ ]:
#nutrient_info
query = '''
SELECT product_code, energy_kcal_value
FROM nutrient_info
ORDER BY energy_kcal_value DESC
LIMIT 10;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT nova_group, AVG(sugars_value) AS avg_sugars
FROM nutrient_info
GROUP BY nova_group;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT COUNT(*) AS high_fat_products
FROM nutrient_info
WHERE fat_value > 20;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT AVG(carbohydrates_value) AS avg_carbohydrates
FROM nutrient_info;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT *
FROM nutrient_info
WHERE sodium_value > 1;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT COUNT(*) AS non_zero_fvn
FROM nutrient_info
WHERE fruits_veg_nuts_estimate_100g > 0;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT *
FROM nutrient_info
WHERE energy_kcal_value > 500;
'''
pd.read_sql(query, conn)

In [ ]:
#derived_metrics

In [ ]:
query = '''
SELECT calorie_category, COUNT(*) AS product_count
FROM derived_metrics
GROUP BY calorie_category;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT COUNT(*) AS high_sugar_count
FROM derived_metrics
WHERE sugar_category = 'High Sugar';
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT AVG(sugar_to_carb_ratio) AS avg_ratio
FROM derived_metrics
WHERE calorie_category = 'High Calorie';
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT *
FROM derived_metrics
WHERE calorie_category = 'High Calorie' AND sugar_category = 'High Sugar';
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT COUNT(*) AS ultra_processed_count
FROM derived_metrics
WHERE is_ultra_processed = TRUE;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT *
FROM derived_metrics
WHERE sugar_to_carb_ratio > 0.7;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT calorie_category, AVG(sugar_to_carb_ratio) AS avg_ratio
FROM derived_metrics
GROUP BY calorie_category;
'''
pd.read_sql(query, conn)

In [ ]:
#Join Queries
query = '''
SELECT pi.brand, COUNT(*) AS high_calorie_count
FROM product_info pi
JOIN derived_metrics dm ON pi.product_code = dm.product_code
WHERE dm.calorie_category = 'High'
GROUP BY pi.brand
ORDER BY high_calorie_count DESC
LIMIT 5;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT dm.calorie_category, AVG(ni.energy_kcal_value) AS avg_energy
FROM derived_metrics dm
JOIN nutrient_info ni ON dm.product_code = ni.product_code
GROUP BY dm.calorie_category;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT pi.brand, COUNT(*) AS ultra_processed_count
FROM product_info pi
JOIN derived_metrics dm ON pi.product_code = dm.product_code
WHERE dm.is_ultra_processed = TRUE
GROUP BY pi.brand;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT pi.product_code, pi.brand, pi.product_name
FROM product_info pi
JOIN derived_metrics dm ON pi.product_code = dm.product_code
WHERE dm.calorie_category = 'High' AND dm.sugar_category = 'High Sugar';
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT pi.brand, AVG(ni.sugars_value) AS avg_sugar
FROM product_info pi
JOIN nutrient_info ni ON pi.product_code = ni.product_code
JOIN derived_metrics dm ON pi.product_code = dm.product_code
WHERE dm.is_ultra_processed = TRUE
GROUP BY pi.brand;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT dm.calorie_category, COUNT(*) AS count_with_fvn
FROM nutrient_info ni
JOIN derived_metrics dm ON ni.product_code = dm.product_code
WHERE ni.fruits_veg_nuts_estimate_100g > 0
GROUP BY dm.calorie_category;
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT dm.product_code, dm.sugar_to_carb_ratio, dm.calorie_category, dm.sugar_category
FROM derived_metrics dm
ORDER BY dm.sugar_to_carb_ratio DESC
LIMIT 5;
'''
pd.read_sql(query, conn)